# Model validation using offline batch inference

<div align="left">
<a target="_blank" href="https://console.anyscale.com/"><img src="https://img.shields.io/badge/🚀 Run_on-Anyscale-9hf"></a>&nbsp;
<a href="https://github.com/anyscale/e2e-xgboost" role="button"><img src="https://img.shields.io/static/v1?label=&amp;message=View%20On%20GitHub&amp;color=586069&amp;logo=github&amp;labelColor=2f363d"></a>&nbsp;
</div>

This tutorial executes a batch inference workload that connects the following heterogeneous workloads:
- Distributed read from cloud storage
- Distributed preprocessing
- Parallel batch inference
- Distributed aggregation of summary metrics

Note that this tutorial fetches the pre-trained model artifacts from the [Distributed training of an XGBoost model](./01-Distributed_Training.ipynb) tutorial.

<img src="https://raw.githubusercontent.com/anyscale/e2e-xgboost/refs/heads/main/images/batch_inference.png" width=800>

The preceding figure illustrates how Ray Data can concurrently process different chunks of data at various stages of the pipeline. This parallel execution maximizes resource utilization and throughput.

Note that this diagram is a simplification for various reasons:

* Backpressure mechanisms may throttle upstream operators to prevent overwhelming downstream stages
* Dynamic repartitioning often occurs as data moves through the pipeline, changing block counts and sizes
* Available resources change as the cluster autoscales
* System failures may disrupt the clean sequential flow shown in the diagram

<div class="alert alert-block alert"> <b> Ray Data streaming execution</b> 

❌ **Traditional batch execution**, non-streaming like Spark without pipelining and SageMaker Batch Transform:
- Reads the entire dataset into memory or a persistent intermediate format
- Only then starts applying transformations like .map, .filter, etc.
- Higher memory pressure and startup latency

✅ **Streaming execution** with Ray Data:
- Starts processing chunks ("blocks") as they're loaded without needing to wait for entire dataset to load
- Reduces memory footprint, no out-of-memory errors, and speeds up time to first output
- Increases resource utilization by reducing idle time
- Enables online-style inference pipelines with minimal latency

<img src="https://raw.githubusercontent.com/anyscale/e2e-xgboost/refs/heads/main/images/streaming.gif" width=700>

**Note**: Ray Data isn't a real-time stream processing engine like Flink or Kafka Streams. Instead, it's batch processing with streaming execution, which is especially useful for iterative ML workloads, ETL pipelines, and preprocessing before training or inference. Ray typically has a [**2-17x throughput improvement**](https://www.anyscale.com/blog/offline-batch-inference-comparing-ray-apache-spark-and-sagemaker#-results-of-throughput-from-experiments) over solutions like Spark and SageMaker Batch Transform.


In [ ]:
%load_ext autoreload
%autoreload all

In [ ]:
# Enable importing from dist_xgboost package.
import os
import sys

sys.path.append(os.path.abspath(".."))

In [ ]:
# Enable Ray Train v2.
os.environ["RAY_TRAIN_V2_ENABLED"] = "1"
# Now it's safe to import from ray.train.

In [ ]:
import ray
import dist_xgboost

# Initialize Ray with the dist_xgboost package.
ray.init(runtime_env={"py_modules": [dist_xgboost]})

# Configure Ray Data logging.
ray.data.DataContext.get_current().enable_progress_bars = False
ray.data.DataContext.get_current().print_on_execution_start = False

## Validating the model using Ray Data
The previous tutorial, [Distributed Training with XGBoost](./01-Distributed_Training.ipynb), trained an XGBoost model and stored it in the MLflow artifact storage. In this step, use it to make predictions on the hold-out test set.



### Data ingestion

Load the test dataset using the same procedure as before:

In [ ]:
from ray.data import Dataset


def prepare_data() -> tuple[Dataset, Dataset, Dataset]:
    """Load and split the dataset into train, validation, and test sets."""
    dataset = ray.data.read_csv("s3://anonymous@air-example-data/breast_cancer.csv")
    seed = 42
    train_dataset, rest = dataset.train_test_split(test_size=0.3, shuffle=True, seed=seed)
    # 15% for validation, 15% for testing.
    valid_dataset, test_dataset = rest.train_test_split(test_size=0.5, shuffle=True, seed=seed)
    return train_dataset, valid_dataset, test_dataset


_, _, test_dataset = prepare_data()
# Use `take()` to trigger execution because Ray Data uses lazy evaluation mode.
test_dataset.take(1)

2025-04-16 21:14:42,328	INFO worker.py:1660 -- Connecting to existing Ray cluster at address: 10.0.23.200:6379...
2025-04-16 21:14:42,338	INFO worker.py:1843 -- Connected to Ray cluster. View the dashboard at https://session-1kebpylz8tcjd34p4sv2h1f9tg.i.anyscaleuserdata.com 
2025-04-16 21:14:42,343	INFO packaging.py:575 -- Creating a file package for local module '/home/ray/default/e2e-xgboost/dist_xgboost'.
2025-04-16 21:14:42,346	INFO packaging.py:367 -- Pushing file package 'gcs://_ray_pkg_fbb1935a37eb6438.zip' (0.02MiB) to Ray cluster...
2025-04-16 21:14:42,347	INFO packaging.py:380 -- Successfully pushed file package 'gcs://_ray_pkg_fbb1935a37eb6438.zip'.
2025-04-16 21:14:42,347	INFO packaging.py:367 -- Pushing file package 'gcs://_ray_pkg_534f9f38a44d4c15f21856eec72c3c338db77a6b.zip' (0.08MiB) to Ray cluster...
2025-04-16 21:14:42,348	INFO packaging.py:380 -- Successfully pushed file package 'gcs://_ray_pkg_534f9f38a44d4c15f21856eec72c3c338db77a6b.zip'.
2025-04-16 21:14:44,609	IN

[{'mean radius': 15.7,
  'mean texture': 20.31,
  'mean perimeter': 101.2,
  'mean area': 766.6,
  'mean smoothness': 0.09597,
  'mean compactness': 0.08799,
  'mean concavity': 0.06593,
  'mean concave points': 0.05189,
  'mean symmetry': 0.1618,
  'mean fractal dimension': 0.05549,
  'radius error': 0.3699,
  'texture error': 1.15,
  'perimeter error': 2.406,
  'area error': 40.98,
  'smoothness error': 0.004626,
  'compactness error': 0.02263,
  'concavity error': 0.01954,
  'concave points error': 0.009767,
  'symmetry error': 0.01547,
  'fractal dimension error': 0.00243,
  'worst radius': 20.11,
  'worst texture': 32.82,
  'worst perimeter': 129.3,
  'worst area': 1269.0,
  'worst smoothness': 0.1414,
  'worst compactness': 0.3547,
  'worst concavity': 0.2902,
  'worst concave points': 0.1541,
  'worst symmetry': 0.3437,
  'worst fractal dimension': 0.08631,
  'target': 0}]

<div class="alert alert-block alert"> <b>💡 Ray Data best practices</b>

- **Use `materialize()` during development**: The `materialize()` method executes and stores the dataset in Ray's shared memory object store. This behavior creates a checkpoint so future operations can start from this point instead of rerunning all operations from scratch.
- **Choose appropriate shuffling strategies**: Ray Data provides various [shuffling strategies](https://docs.ray.io/en/latest/data/shuffling-data.html) including local shuffles and per-epoch shuffles. You need to shuffle this dataset because the original data groups items by class.


Next, transform the input data the same way you did during training. Fetch the preprocessor from the artifact registry:

In [ ]:
import pickle

from dist_xgboost.constants import preprocessor_fname
from dist_xgboost.data import get_best_model_from_registry

best_run, best_artifacts_dir = get_best_model_from_registry()

with open(os.path.join(best_artifacts_dir, preprocessor_fname), "rb") as f:
    preprocessor = pickle.load(f)

Now define the transformation step in the Ray Data pipeline. Instead of processing each item individually with `.map()`, use Ray Data's [`map_batches`](https://docs.ray.io/en/latest/data/api/doc/ray.data.Dataset.map_batches.html) method to process entire batches at once, which is much more efficient:

In [ ]:
def transform_with_preprocessor(batch_df, preprocessor):
    # The preprocessor doesn't include the `target` column,
    # so remove it temporarily, then add it back.
    target = batch_df.pop("target")
    transformed_features = preprocessor.transform_batch(batch_df)
    transformed_features["target"] = target
    return transformed_features


# Apply the transformation to each batch.
test_dataset = test_dataset.map_batches(
    transform_with_preprocessor,
    fn_kwargs={"preprocessor": preprocessor},
    batch_format="pandas",
    batch_size=1000,
)

test_dataset.show(1)

{'mean radius': 0.4202879281965173, 'mean texture': 0.2278148207774012, 'mean perimeter': 0.35489846800755104, 'mean area': 0.29117590184541364, 'mean smoothness': -0.039721410464208406, 'mean compactness': -0.30321758777095337, 'mean concavity': -0.2973304995033593, 'mean concave points': 0.05629912285695481, 'mean symmetry': -0.6923528276633714, 'mean fractal dimension': -1.0159489469979848, 'radius error': -0.1244372811541358, 'texture error': -0.1073358496664629, 'perimeter error': -0.2253381140213174, 'area error': 0.001804996358367429, 'smoothness error': -0.8087740189276656, 'compactness error': -0.1437977993323648, 'concavity error': -0.3926326901399853, 'concave points error': -0.34157926393517024, 'symmetry error': -0.5862955941365042, 'fractal dimension error': -0.496152478599194, 'worst radius': 0.7695260874215265, 'worst texture': 1.1287525414418031, 'worst perimeter': 0.6310282171135395, 'worst area': 0.6506421499178707, 'worst smoothness': 0.39052158034274154, 'worst com

### Load the trained model

Now that you've defined the preprocessing pipeline, you're ready to run batch inference. Load the model from the artifact registry:

In [ ]:
from ray.train import Checkpoint
from ray.train.xgboost import RayTrainReportCallback

checkpoint = Checkpoint.from_directory(best_artifacts_dir)
model = RayTrainReportCallback.get_model(checkpoint)

### Run batch inference

Next, run the inference step. To avoid repeatedly loading the model for each batch, define a reusable class that can use the same XGBoost model for different batches:

In [ ]:
import pandas as pd
import xgboost

from dist_xgboost.data import load_model_and_preprocessor


class Validator:
    def __init__(self):
        _, self.model = load_model_and_preprocessor()

    def __call__(self, batch: pd.DataFrame) -> pd.DataFrame:
        # Remove the target column for inference.
        target = batch.pop("target")
        dmatrix = xgboost.DMatrix(batch)
        predictions = self.model.predict(dmatrix)

        results = pd.DataFrame({"prediction": predictions, "target": target})
        return results

Now parallelize inference across replicas of the model by processing data in batches:

In [ ]:
test_predictions = test_dataset.map_batches(
    Validator,
    concurrency=4,  # Number of model replicas.
    batch_format="pandas",
)

test_predictions.show(1)

2025-04-16 21:14:56,496	WARNING actor_pool_map_operator.py:287 -- To ensure full parallelization across an actor pool of size 4, the Dataset should consist of at least 4 distinct blocks. Consider increasing the parallelism when creating the Dataset.


{'prediction': 0.031001044437289238, 'target': 0}


### Calculate evaluation metrics

Now that you have predictions, evaluate the model's accuracy, precision, recall, and F1-score. Calculate the number of true positives, true negatives, false positives, and false negatives across the test subset:

In [ ]:
from sklearn.metrics import confusion_matrix


def confusion_matrix_batch(batch, threshold=0.5):
    # Apply a threshold to get binary predictions.
    batch["prediction"] = (batch["prediction"] > threshold).astype(int)

    result = {}
    cm = confusion_matrix(batch["target"], batch["prediction"], labels=[0, 1])
    result["TN"] = cm[0, 0]
    result["FP"] = cm[0, 1]
    result["FN"] = cm[1, 0]
    result["TP"] = cm[1, 1]
    return pd.DataFrame(result, index=[0])


test_results = test_predictions.map_batches(confusion_matrix_batch, batch_format="pandas", batch_size=1000)

Finally, aggregate the confusion matrix results from all batches to get the global counts. This step materializes the dataset and executes all previously declared lazy transformations:

In [ ]:
# Sum all confusion matrix values across batches.
cm_sums = test_results.sum(["TN", "FP", "FN", "TP"])

# Extract confusion matrix components.
tn = cm_sums["sum(TN)"]
fp = cm_sums["sum(FP)"]
fn = cm_sums["sum(FN)"]
tp = cm_sums["sum(TP)"]

# Calculate metrics.
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

metrics = {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

2025-04-16 21:15:01,144	WARNING actor_pool_map_operator.py:287 -- To ensure full parallelization across an actor pool of size 4, the Dataset should consist of at least 4 distinct blocks. Consider increasing the parallelism when creating the Dataset.


In [ ]:
print("Validation results:")
for key, value in metrics.items():
    print(f"{key}: {value:.4f}")

Validation results:
precision: 0.9464
recall: 0.9815
f1: 0.9636
accuracy: 0.9535


The following is the expected output:

```
Validation results:
precision: 0.9574
recall: 1.0000
f1: 0.9783
accuracy: 0.9767
```

## Observability

Ray Data provides built-in observability features to help you monitor and debug data processing pipelines:
 
<img src="https://raw.githubusercontent.com/anyscale/e2e-xgboost/refs/heads/main/images/ray_data_observability.png" width=800>


### Production deployment

You can wrap the training workload as a production-grade [Anyscale Job](https://docs.anyscale.com/platform/jobs/). See the [API ref](https://docs.anyscale.com/reference/job-api/):

```bash
# Production batch job.
anyscale job submit --name=validate-xboost-breast-cancer-model \
  --containerfile="${WORKING_DIR}/containerfile" \
  --working-dir="${WORKING_DIR}" \
  --exclude="" \
  --max-retries=0 \
  -- python dist_xgboost/infer.py
```

Note that in order for this command to succeed, first configure MLflow to store the artifacts in storage that's readable across clusters. Anyscale offers a variety of storage options that work out of the box, such as a [default storage bucket](https://docs.anyscale.com/configuration/storage/#anyscale-default-storage-bucket), as well as [automatically mounted network storage](https://docs.anyscale.com/configuration/storage/) shared at the cluster, user, and cloud levels. You could also set up your own network mounts or storage buckets.

## Summary

In this tutorial, you:

1. Loaded a test dataset using distributed reads from cloud storage
2. Transformed the dataset in a streaming fashion with the same preprocessor used during training
3. Treated a validation pipeline to:
   - Make predictions on the test data using multiple model replicas
   - Calculate confusion matrix components for each batch
   - Aggregate results across all batches
4. Computed key performance metrics, like precision, recall, F1-score, and accuracy

The same code can efficiently run on terabyte-scale datasets without modifications using Ray Data's distributed processing capabilities.

The next tutorial shows how to serve this XGBoost model for online inference using Ray Serve.